In [6]:
import requests
import pandas as pd
import time
import re
import csv
import os
from bs4 import BeautifulSoup

import sys
sys.path.append("..") # 親ディレクトリを追加
from module.random_agent import RandomUserAgent
from module.path_reader import PathReader

### 関数定義

In [7]:
# what: 任意の期間(20xx/xx~20yy/yy)を入力するとその期間の年月をリストで返してくれる関数
# for:  スクレイピング期間を求めるため
def month_list(from_year: int, from_month: int, to_year: int, to_month: int):
    result = []
    year, month = from_year, from_month
    while (year < to_year) or (year == to_year and month <= to_month):
        result.append([year, month])
        # 月を1つ進める
        month += 1
        if month > 12:
            month = 1
            year += 1
    return result

In [14]:
# netkeiba.comの開催日程のページからレース開催日と全てのレースIDを取得する関数
def get_monthly_race_ids(year: int, month: int, sleep: float = 0.8):
    """
    指定した年・月の全レースIDを取得して DataFrame を返す。
    - year: 2024
    - month: 9  (1〜12)
    """
    ua = RandomUserAgent() # ランダムなUser-Agentを取得
    calendar_url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
    resp = requests.get(calendar_url, headers={"User-Agent": ua.user_agent})
    resp.encoding = resp.apparent_encoding or "EUC-JP"
    soup = BeautifulSoup(resp.text, "html.parser")

    # カレンダーページから kaisai_date=YYYYMMDD を抽出（重複除去）
    date_vals = sorted(set(re.findall(r"kaisai_date=(\d{8})", str(soup))))

    all_race_ids = []
    all_dates = []

    for date in date_vals:
        try:
            # 日付ごとのレース一覧ページ（sub を使うのが確実）
            race_list_url = f"https://race.netkeiba.com/top/race_list_sub.html?kaisai_date={date}"
            r = requests.get(race_list_url, headers={"User-Agent": ua.user_agent})
            r.encoding = r.apparent_encoding or "EUC-JP"
            soup_list = BeautifulSoup(r.text, "html.parser")

            # ページ内の race_id= を持つリンクから race_id を抽出
            ids = set(re.findall(r"race_id=(\d{12})", str(soup_list)))
            for rid in ids:
                all_race_ids.append(rid)
                all_dates.append(date)

        except Exception as e:
            print(f"[Warning] {date} の取得でエラー: {e}")
        time.sleep(sleep)  # マナーとして間隔をあける

    df = pd.DataFrame({"date": all_dates, "race_id": all_race_ids})
    return df.drop_duplicates().reset_index(drop=True)

In [15]:
# what: race_id_list.csvのファイル名に日付を記載しておく関数
# for:  どの期間のrace_idを取得したか一目でわかるようにするため
# in:   ファイルに書かれているキーワード
# out:  更新したファイルの名前
def csv_name_update(partial_name: str, folder_path: str = None) -> str:
    # 部分一致するCSVファイルを探す
    matched_files = [
        f for f in os.listdir(folder_path)
        if f.endswith(".csv") and partial_name in f
    ]

    # 最初に見つかったファイルを original_filename に代入
    if matched_files:
        original_filename = matched_files[0]
        original_path = os.path.join(folder_path, original_filename)
        # print(f"見つかったファイル: {original_filename}")
    else:
        print("一致するCSVファイルが見つかりませんでした。")

    # 新しいファイル名を構築するための値を抽出
    with open(original_path, newline='', encoding='utf-8') as f:
        reader = list(csv.reader(f))
        
        # 2行目1列目の値（インデックスは1, 0）
        second_row_value = reader[1][0]
        part1 = second_row_value[2:6]  # 3文字目〜6文字目（0-indexed）

        # 最終行1列目の値
        last_row_value = reader[-1][0]
        part2 = last_row_value[2:6]

    # 新しいファイル名の構築
    # base_name, ext = os.path.splitext(original_filename)
    new_filename = f"{partial_name}_{part1}_{part2}.csv"
    new_path = os.path.join(folder_path, new_filename)

    # ファイル名の変更
    os.rename(original_path, new_path)

    print(f"Currently File name: {new_filename}")
    return new_path

### 実行関数：レースIDの取得

In [ ]:
# 入力パラメータ
# 実行環境(NotePC/Desktop)の選択
reader = PathReader("../file_path_NotePC.json") # NotePC用
# reader = PathReader("../file_path_Desktop.json") # Desktop用

# データの取得期間の設定
from_year = 2023
from_month = 1
to_year = 2023
to_month = 9

In [ ]:
# スクレイピング期間の取得
scraping_dates = month_list(from_year, from_month, to_year, to_month)
# csvの名前を一目で見てわかるようにファイル名に日付を入れる
race_id_list_path = csv_name_update(partial_name="race_id_list", folder_path=reader.get_path("data_folder"))
all_new_ids = []
# 任意の期間の全レースのrace_idを取得
for year, month in scraping_dates:
    try:
        df_new = get_monthly_race_ids(year, month, sleep=1.0)
        all_new_ids.append(df_new)
        print(f"{year}-{month} 取得件数: {len(df_new)}")
    except Exception as e:
        print(f"Error fetching {year}-{month}: {e}")

if len(all_new_ids) > 0:
    scrape_race_id_list = pd.concat(all_new_ids, ignore_index=True)
else:
    scrape_race_id_list = pd.DataFrame(columns=["data", "race_id"])

if os.path.exists(race_id_list_path):
    race_id_list_old = pd.read_csv(race_id_list_path) # 既存データを読み込み
    race_id_list = pd.concat([race_id_list_old, scrape_race_id_list]) # データの結合
    race_id_list = race_id_list.drop_duplicates(subset=["race_id", "date"], keep="first") # 重複race_idを削除
else:
    race_id_list = scrape_race_id_list
if "date" in race_id_list.columns:
    race_id_list["date"] = race_id_list["date"].astype(int)
    race_id_list = race_id_list.sort_values(by="date", ascending=True)
    race_id_list["date"] = race_id_list["date"].astype(str)
race_id_list.to_csv(race_id_list_path, index=False, encoding="utf-8")

# csvの名前を一目で見てわかるようにファイル名に日付を入れる
race_id_list_path = csv_name_update(partial_name="race_id_list", folder_path=reader.get_path("data_folder"))

Currently File name: race_id_list_2310_2404.csv
2023-1 取得件数: 312
2023-2 取得件数: 288
2023-3 取得件数: 264
2023-4 取得件数: 336
2023-5 取得件数: 264
2023-6 取得件数: 264
2023-7 取得件数: 336
2023-8 取得件数: 264
2023-9 取得件数: 264
Currently File name: race_id_list_2301_2404.csv
